In [38]:
pip install lxml

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ------------------------------- -------- 3.1/4.0 MB 19.8 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 16.8 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium webdriver-manager beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


BELOW THERE IS "EXTRAFILTERED"

In [10]:
# evergabe_singlepass_incl_excl_multilang.py
# FAST single-pass scraper with multilingual inclusion & exclusion filters,
# full column extraction, and per-cell link capture.

import csv, time, re, math, datetime, unicodedata
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

START_URL = "https://www.evergabe-online.de/search.html?4"

# ---------------- INCLUSION KEYWORDS (DE/FR/IT/EN) ----------------
RAW_INCLUDE = [
    # Study / Studie
    "study", "studie", "étude", "etude", "studio",

    # Analysis / Analyse
    "analysis", "analyse", "analisi",

    # Economy / Wirtschaft / Ökonomie
    "economy", "economics", "wirtschaft", "ökonomie", "economie", "economia",

    # Benchmarking
    "benchmark", "benchmarking",

    # Wirtschaftsberatung (economic/business consulting)
    "wirtschaftsberatung", "consulting", "conseil", "consulenza",

    # CPV & categories
    "72000000", "79300000",
    "it-dienste",
    "beratung, software-entwicklung, internet und hilfestellung",
    "markt- und wirtschaftsforschung", "umfragen und statistiken",

    # Dienstleistung
    "dienstleistung", "service", "prestation", "prestazione",

    # Ausschreibung
    "ausschreibung", "appel d'offres", "appels d'offres", "gara", "procurement",

    # Offenes Verfahren
    "offenes verfahren", "procédure ouverte", "procedure ouverte", "procedura aperta",

    # BKS / Bundesamt für Statistik / OFS
    "bks",
    "bundesamt für statistik",
    "office fédéral de la statistique", "office federal de la statistique",
    "ufficio federale di statistica",
    "federal statistical office", "ofs", "bfs",

    # SECO / Staatssekretariat für Wirtschaft
    "seco", "staatssekretariat für wirtschaft",
    "secrétariat d'état à l'économie", "secretariat d'etat a l'economie",
    "segretariato di stato dell'economia",

    # Cantons / regions (Zürich, Luzern, generic canton/region)
    "kanton zürich", "kanton zuerich", "canton de zurich", "cantone zurigo",
    "kanton luzern", "canton de lucerne", "cantone lucerna",
    "kanton", "canton", "cantone",
    "region", "regionen", "région", "regione",

    # Bundesamt für… (generic)
    "bundesamt für", "bundesamt fuer",

    # BBL
    "bundesamt für bauten und logistik", "bundesamt fuer bauten und logistik", "bbl",

    # Index (e.g. Verbraucherindex)
    "index", "verbraucherindex", "price index", "indice des prix", "indice dei prezzi",

    # Wirtschaftsforschung
    "wirtschaftsforschung", "economic research", "recherche économique",
    "recherche economique", "ricerca economica",
]

# ---------------- EXCLUSION KEYWORDS (DE/FR/IT/EN) ----------------
RAW_EXCLUDE = [
    "construction", "bau", "bâtiment", "batiment", "costruzioni",
    "health care", "gesundheit", "santé", "sante", "sanità", "sanita",
    "transport", "verkehr", "transports", "trasporti",
    "mobility", "mobilität", "mobilite", "mobilité", "mobilita",
    "sport",  # same in most langs
    "culture", "kultur", "culture", "cultura",
    "street", "strasse", "straße", "rue", "strada",
    "infrastructure", "infrastruktur", "infrastructure", "infrastruttura",
    "process", "prozess", "processus", "processo",
    "it", "informatik", "informatique", "informatica",
]

def _norm(s: str) -> str:
    """lowercase, strip accents, collapse spaces"""
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    return re.sub(r"\s+", " ", s.strip().lower())

INCLUDE = {_norm(k) for k in RAW_INCLUDE if k.strip()}
EXCLUDE = {_norm(k) for k in RAW_EXCLUDE if k.strip()}

# ---------------- Selenium setup ----------------
def make_driver(headless=True):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--no-sandbox"); opts.add_argument("--disable-gpu")
    opts.add_argument("--lang=de-DE")
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=opts)

def accept_cookies(driver):
    labels = ["Alle akzeptieren","Akzeptieren","Zustimmen","Einverstanden","OK","Okay","Ich stimme zu"]
    for label in labels:
        try:
            btn = WebDriverWait(driver, 4).until(
                EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{label}')]"))
            )
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)
            break
        except Exception:
            pass

def wait_for_table(driver, timeout=20):
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody tr"))
    )

def first_cell_token(driver):
    try:
        return driver.find_element(By.CSS_SELECTOR, "table tbody tr td").get_attribute("innerText").strip()
    except Exception:
        return None

def set_page_size_max(driver):
    # Try <select> page-size
    tried = False
    for sel in ["select", "select.page-size", "select[name*='size' i]"]:
        try:
            el = driver.find_element(By.CSS_SELECTOR, sel)
            s = Select(el)
            labels = [o.text.strip() or o.get_attribute("value") for o in s.options]
            target = None
            for want in ("100","200","Alle","Alles"):
                for lbl in labels:
                    if want in lbl:
                        target = lbl; break
                if target: break
            if not target and labels: target = labels[-1]
            if target:
                old = first_cell_token(driver)
                s.select_by_visible_text(target)
                WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != old)
                tried = True
                break
        except Exception:
            pass
    # Try toolbar buttons (10|25|50|100)
    if not tried:
        for want in ("100","200"):
            try:
                btn = driver.find_element(By.XPATH, f"//button[normalize-space()='{want}']")
                old = first_cell_token(driver)
                driver.execute_script("arguments[0].click();", btn)
                WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != old)
                break
            except Exception:
                pass

# ---------------- Full column extraction + filters ----------------
def unique_headers(headers):
    counts = defaultdict(int); out = []
    for h in headers:
        key = h or "col"; counts[key] += 1
        out.append(key if counts[key] == 1 else f"{key}_{counts[key]}")
    return out

def get_table_headers(driver):
    table = driver.find_element(By.CSS_SELECTOR, "table")
    ths = table.find_elements(By.CSS_SELECTOR, "thead th")
    headers = [th.get_attribute("innerText").strip() or th.get_attribute("title") or th.get_attribute("aria-label") or "" for th in ths]
    if not any(h for h in headers):
        try:
            first_row = table.find_element(By.CSS_SELECTOR, "tbody tr")
            tds = first_row.find_elements(By.CSS_SELECTOR, "td")
            tmp = []
            for td in tds:
                label = td.get_attribute("data-title") or td.get_attribute("aria-label") or ""
                tmp.append(label)
            headers = tmp
        except Exception:
            pass
    if not headers:
        first_row = table.find_element(By.CSS_SELECTOR, "tbody tr")
        n = len(first_row.find_elements(By.CSS_SELECTOR, "td"))
        headers = [f"col_{i+1}" for i in range(n)]
    return unique_headers([h.strip() for h in headers])

def row_dicts_all_columns(driver, headers):
    """
    Extract all visible columns and apply inclusion/exclusion filters.
    Adds <column>_link if a link exists in that cell.
    """
    table = driver.find_element(By.CSS_SELECTOR, "table")
    rows = []
    for tr in table.find_elements(By.CSS_SELECTOR, "tbody tr"):
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        if not tds: continue
        row = {}; concat = []
        for i, td in enumerate(tds):
            key = headers[i] if i < len(headers) else f"col_{i+1}"
            text = td.get_attribute("innerText").strip()
            row[key] = text
            concat.append(text)
            try:
                a = td.find_element(By.CSS_SELECTOR, "a[href]")
                row[f"{key}_link"] = a.get_attribute("href")
            except Exception:
                pass
        norm = _norm(" | ".join(concat))
        hits_incl = sorted({kw for kw in INCLUDE if kw in norm})
        hits_excl = sorted({kw for kw in EXCLUDE if kw in norm})
        if hits_incl and not hits_excl:
            row["matched_keywords"] = ", ".join(hits_incl)
            # also store exclusions hit (for QA); empty in accepted rows
            row["excluded_keywords_hit"] = ", ".join(hits_excl)
            rows.append(row)
    return rows

def go_next(driver):
    try:
        nxt = driver.find_element(By.CSS_SELECTOR, "a[rel='next']")
        if (nxt.get_attribute("aria-disabled") or "").lower() == "true":
            return False
        token = first_cell_token(driver)
        driver.execute_script("arguments[0].click();", nxt)
        WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != token)
        return True
    except Exception:
        return False

def crawl_all_once(headless=True, max_pages=None, per_page_delay=0.15):
    driver = make_driver(headless=headless)
    driver.get(START_URL)
    accept_cookies(driver)
    wait_for_table(driver)
    set_page_size_max(driver)

    # optional: quick info
    try:
        info = driver.find_element(By.XPATH, "//*[contains(., 'Zeige') and contains(., 'von')]").text
        m = re.search(r"Zeige\s+(\d+)\s+bis\s+(\d+)\s+von\s+(\d+)", info)
        if m:
            start, end, total = map(int, m.groups()); size = end - start + 1
            pages = math.ceil(total / size) if size else None
            if pages: print(f"ℹ️ ~{pages} pages, page size {size}")
    except Exception:
        pass

    headers = get_table_headers(driver)
    results, page = [], 1
    while True:
        print(f"📄 Page {page} …", end="", flush=True)
        page_rows = row_dicts_all_columns(driver, headers)
        results.extend(page_rows)
        print(f" kept {len(page_rows)}")
        if max_pages and page >= max_pages:
            break
        time.sleep(per_page_delay)
        if not go_next(driver):
            break
        page += 1

    driver.quit()

    # Save CSV
    all_keys = set(headers + [f"{h}_link" for h in headers] + ["matched_keywords", "excluded_keywords_hit"])
    for r in results: all_keys.update(r.keys())
    preferred = ["Bezeichnung","Geschäftszeichen","Vergabestelle","Ort der Leistung","matched_keywords","excluded_keywords_hit"]
    header = [k for k in preferred if k in all_keys] + [k for k in sorted(all_keys) if k not in preferred]

    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    out = f"evergabe_filtered_multilang_{ts}.csv"
    with open(out, "w", newline="", encoding="utf-8-sig") as f:
        w = csv.DictWriter(f, fieldnames=header)
        w.writeheader(); w.writerows(results)
    print(f"✅ Saved {len(results)} rows to {out}")

if __name__ == "__main__":
    # headless=False to watch it work. Set max_pages for a quick test run.
    crawl_all_once(headless=True, max_pages=None, per_page_delay=0.15)


ℹ️ ~127 pages, page size 10
📄 Page 1 … kept 4
📄 Page 2 … kept 2
📄 Page 3 … kept 2
📄 Page 4 … kept 2
📄 Page 5 … kept 1
📄 Page 6 … kept 1
📄 Page 7 … kept 2
📄 Page 8 … kept 2
📄 Page 9 … kept 2
📄 Page 10 … kept 1
📄 Page 11 … kept 2
📄 Page 12 … kept 0
📄 Page 13 … kept 2
📄 Page 14 … kept 2
📄 Page 15 … kept 2
📄 Page 16 … kept 5
📄 Page 17 … kept 4
📄 Page 18 … kept 5
📄 Page 19 … kept 1
📄 Page 20 … kept 2
📄 Page 21 … kept 4
📄 Page 22 … kept 1
📄 Page 23 … kept 3
📄 Page 24 … kept 2
📄 Page 25 … kept 3
📄 Page 26 … kept 1
📄 Page 27 … kept 2
📄 Page 28 … kept 2
📄 Page 29 … kept 1
📄 Page 30 … kept 1
📄 Page 31 … kept 0
📄 Page 32 … kept 4
📄 Page 33 … kept 3
📄 Page 34 … kept 1
📄 Page 35 … kept 2
📄 Page 36 … kept 1
📄 Page 37 … kept 1
📄 Page 38 … kept 2
📄 Page 39 … kept 2
📄 Page 40 … kept 3
📄 Page 41 … kept 3
📄 Page 42 … kept 1
📄 Page 43 … kept 2
📄 Page 44 … kept 2
📄 Page 45 … kept 2
📄 Page 46 … kept 4
📄 Page 47 … kept 0
📄 Page 48 … kept 2
📄 Page 49 … kept 0
📄 Page 50 … kept 2
📄 Page 51 … kept 1
📄 Page 52 … 

THIS IS IN GENERAL

In [11]:
# FAST single-pass scraper with FULL columns
# Keywords limited to user's list
# pip install selenium webdriver-manager

import csv, time, re, math, datetime, unicodedata
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

START_URL = "https://www.evergabe-online.de/search.html?4"

# ---------------- KEYWORDS (ONLY THE ONES YOU GAVE) ----------------
RAW_KW = [
    # Study / Studie
    "Study", "Studie",

    # Analysis / Analyse
    "Analysis", "Analyse",

    # Economy / Wirtschaft / Ökonomie
    "Economy", "Wirtschaft", "Ökonomie",

    # Benchmarking
    "Benchmarking",

    # Wirtschaftsberatung
    "Wirtschaftsberatung",

    # Haupt-CPV … (use codes & main German phrases appearing with them)
    "72000000", "79300000",
    "IT-Dienste",  # aus "IT-Dienste: Beratung, Software-Entwicklung, Internet und Hilfestellung"
    "Beratung, Software-Entwicklung, Internet und Hilfestellung",
    "Beratung, Markt- und Wirtschaftsforschung; Umfragen und Statistiken",
    "Markt- und Wirtschaftsforschung; Umfragen und Statistiken",

    # Dienstleistung
    "Dienstleistung",

    # Ausschreibung
    "Ausschreibung",

    # Offenes Verfahren
    "Offenes Verfahren",

    # BKS / Bundesamt für Statistik, Office fédéral de la statistique
    "BKS",
    "Bundesamt für Statistik",
    "Office fédéral de la statistique",

    # SECO / Staatssekretariat für Wirtschaft
    "SECO",
    "Staatssekretariat für Wirtschaft",

    # Kanton Zürich, Luzern etc…
    "Kanton Zürich", "Zürich",
    "Kanton Luzern", "Luzern",

    # Bundesamt für Bauten und Logistik BBL
    "Bundesamt für Bauten und Logistik",
    "BBL",

    # In general: Bundesamt für..
    "Bundesamt für",

    # Index (e.g. Verbraucherindex)
    "Index", "Verbraucherindex",

    # Regionen
    "Regionen",

    # Wirtschaftsforschung
    "Wirtschaftsforschung",
]

def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    return re.sub(r"\s+"," ", s.strip().lower())

KEYWORDS = {_norm(k) for k in RAW_KW if k.strip()}

# ---------------- Selenium setup ----------------
def make_driver(headless=True):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--no-sandbox"); opts.add_argument("--disable-gpu")
    opts.add_argument("--lang=de-DE")
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=opts)

def accept_cookies(driver):
    for label in ["Alle akzeptieren","Akzeptieren","Zustimmen","Einverstanden","OK","Okay","Ich stimme zu"]:
        try:
            btn = WebDriverWait(driver, 4).until(
                EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{label}')]"))
            )
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)
            break
        except Exception:
            pass

def wait_for_table(driver, timeout=20):
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody tr"))
    )

def first_cell_token(driver):
    try:
        return driver.find_element(By.CSS_SELECTOR, "table tbody tr td").get_attribute("innerText").strip()
    except Exception:
        return None

def set_page_size_max(driver):
    tried = False
    for sel in ["select", "select.page-size", "select[name*='size' i]"]:
        try:
            el = driver.find_element(By.CSS_SELECTOR, sel)
            s = Select(el)
            labels = [o.text.strip() or o.get_attribute("value") for o in s.options]
            target = None
            for want in ("100","200","Alle","Alles"):
                for lbl in labels:
                    if want in lbl:
                        target = lbl; break
                if target: break
            if not target and labels: target = labels[-1]
            if target:
                old = first_cell_token(driver)
                s.select_by_visible_text(target)
                WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != old)
                tried = True
                break
        except Exception:
            pass
    if not tried:
        for want in ("100","200"):
            try:
                btn = driver.find_element(By.XPATH, f"//button[normalize-space()='{want}']")
                old = first_cell_token(driver)
                driver.execute_script("arguments[0].click();", btn)
                WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != old)
                break
            except Exception:
                pass

# -------- FULL column extraction + per-row keyword match --------
from collections import defaultdict
def unique_headers(headers):
    counts = defaultdict(int)
    out = []
    for h in headers:
        key = h or "col"
        counts[key] += 1
        out.append(key if counts[key] == 1 else f"{key}_{counts[key]}")
    return out

def get_table_headers(driver):
    table = driver.find_element(By.CSS_SELECTOR, "table")
    ths = table.find_elements(By.CSS_SELECTOR, "thead th")
    headers = [th.get_attribute("innerText").strip() or th.get_attribute("title") or th.get_attribute("aria-label") or "" for th in ths]
    if not any(h for h in headers):
        try:
            first_row = table.find_element(By.CSS_SELECTOR, "tbody tr")
            tds = first_row.find_elements(By.CSS_SELECTOR, "td")
            tmp = []
            for td in tds:
                label = td.get_attribute("data-title") or td.get_attribute("aria-label") or ""
                tmp.append(label)
            headers = tmp
        except Exception:
            pass
    if not headers:
        first_row = table.find_element(By.CSS_SELECTOR, "tbody tr")
        n = len(first_row.find_elements(By.CSS_SELECTOR, "td"))
        headers = [f"col_{i+1}" for i in range(n)]
    return unique_headers([h.strip() for h in headers])

def row_dicts_all_columns(driver, headers):
    table = driver.find_element(By.CSS_SELECTOR, "table")
    rows = []
    for tr in table.find_elements(By.CSS_SELECTOR, "tbody tr"):
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        if not tds:
            continue
        row = {}
        concat = []
        for i, td in enumerate(tds):
            key = headers[i] if i < len(headers) else f"col_{i+1}"
            text = td.get_attribute("innerText").strip()
            row[key] = text
            concat.append(text)
            try:
                a = td.find_element(By.CSS_SELECTOR, "a[href]")
                row[f"{key}_link"] = a.get_attribute("href")
            except Exception:
                pass
        norm = _norm(" | ".join(concat))
        hits = [kw for kw in KEYWORDS if kw in norm]
        if hits:
            row["matched_keywords"] = ", ".join(sorted(set(hits)))
            rows.append(row)
    return rows

def go_next(driver):
    try:
        nxt = driver.find_element(By.CSS_SELECTOR, "a[rel='next']")
        if (nxt.get_attribute("aria-disabled") or "").lower() == "true":
            return False
        token = first_cell_token(driver)
        driver.execute_script("arguments[0].click();", nxt)
        WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != token)
        return True
    except Exception:
        return False

def crawl_all_once(headless=True, max_pages=None, per_page_delay=0.15):
    driver = make_driver(headless=headless)
    driver.get(START_URL)
    accept_cookies(driver)
    wait_for_table(driver)
    set_page_size_max(driver)

    # optional info
    try:
        info = driver.find_element(By.XPATH, "//*[contains(., 'Zeige') and contains(., 'von')]").text
        m = re.search(r"Zeige\s+(\d+)\s+bis\s+(\d+)\s+von\s+(\d+)", info)
        if m:
            start, end, total = map(int, m.groups()); size = end - start + 1
            pages = math.ceil(total / size) if size else None
            if pages: print(f"ℹ️ ~{pages} pages with page size {size}")
    except Exception:
        pass

    headers = get_table_headers(driver)
    results, page = [], 1
    while True:
        print(f"📄 Page {page} …", end="", flush=True)
        page_rows = row_dicts_all_columns(driver, headers)
        results.extend(page_rows)
        print(f" kept {len(page_rows)}")
        if max_pages and page >= max_pages:
            break
        time.sleep(per_page_delay)
        if not go_next(driver):
            break
        page += 1

    driver.quit()

    # save CSV
    all_keys = set(headers + [f"{h}_link" for h in headers] + ["matched_keywords"])
    for r in results: all_keys.update(r.keys())
    preferred = ["Bezeichnung","Geschäftszeichen","Vergabestelle","Ort der Leistung"]
    header = [k for k in preferred if k in all_keys] + [k for k in sorted(all_keys) if k not in preferred]

    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    out = f"evergabe_keywords_fullcolumns_{ts}.csv"
    with open(out, "w", newline="", encoding="utf-8-sig") as f:
        w = csv.DictWriter(f, fieldnames=header)
        w.writeheader(); w.writerows(results)
    print(f"✅ Saved {len(results)} rows to {out}")

if __name__ == "__main__":
    # headless=False to watch; set max_pages for a quick test
    crawl_all_once(headless=True, max_pages=None, per_page_delay=0.15)


ℹ️ ~127 pages with page size 10
📄 Page 1 … kept 10
📄 Page 2 … kept 10
📄 Page 3 … kept 10
📄 Page 4 … kept 9
📄 Page 5 … kept 9
📄 Page 6 … kept 10
📄 Page 7 … kept 10
📄 Page 8 … kept 10
📄 Page 9 … kept 10
📄 Page 10 … kept 10
📄 Page 11 … kept 10
📄 Page 12 … kept 10
📄 Page 13 … kept 9
📄 Page 14 … kept 10
📄 Page 15 … kept 10
📄 Page 16 … kept 10
📄 Page 17 … kept 10
📄 Page 18 … kept 10
📄 Page 19 … kept 10
📄 Page 20 … kept 10
📄 Page 21 … kept 10
📄 Page 22 … kept 8
📄 Page 23 … kept 10
📄 Page 24 … kept 10
📄 Page 25 … kept 10
📄 Page 26 … kept 10
📄 Page 27 … kept 8
📄 Page 28 … kept 8
📄 Page 29 … kept 10
📄 Page 30 … kept 10
📄 Page 31 … kept 9
📄 Page 32 … kept 9
📄 Page 33 … kept 8
📄 Page 34 … kept 9
📄 Page 35 … kept 9
📄 Page 36 … kept 9
📄 Page 37 … kept 10
📄 Page 38 … kept 10
📄 Page 39 … kept 9
📄 Page 40 … kept 10
📄 Page 41 … kept 10
📄 Page 42 … kept 10
📄 Page 43 … kept 10
📄 Page 44 … kept 9
📄 Page 45 … kept 7
📄 Page 46 … kept 9
📄 Page 47 … kept 8
📄 Page 48 … kept 10
📄 Page 49 … kept 10
📄 Page 50 … ke

In [16]:
# evergabe_singlepass_incl_excl_multilang_with_details.py
# FAST single-pass crawl with:
# - multilingual inclusion/exclusion filters
# - full column extraction (+ per-cell links)
# - stable unique id per row
# - visit each detail_url: grab text content and download first PDF
# - CSV output + a /downloads folder with PDFs

import csv, time, re, math, datetime, unicodedata, os, uuid, pathlib
from collections import defaultdict
import requests

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

START_URL = "https://www.evergabe-online.de/search.html?4"
DOWNLOAD_DIR = "downloads"  # PDFs will be saved here

# ---------------- INCLUSION KEYWORDS (DE/FR/IT/EN) ----------------
RAW_INCLUDE = [
    "study", "studie", "étude", "etude", "studio",
    "analysis", "analyse", "analisi",
    "economy", "economics", "wirtschaft", "ökonomie", "economie", "economia",
    "benchmark", "benchmarking",
    "wirtschaftsberatung", "consulting", "conseil", "consulenza",
    "72000000", "79300000",
    "it-dienste",
    "beratung, software-entwicklung, internet und hilfestellung",
    "markt- und wirtschaftsforschung", "umfragen und statistiken",
    "dienstleistung", "service", "prestation", "prestazione",
    "ausschreibung", "appel d'offres", "appels d'offres", "gara", "procurement",
    "offenes verfahren", "procédure ouverte", "procedure ouverte", "procedura aperta",
    "bks", "bundesamt für statistik",
    "office fédéral de la statistique", "office federal de la statistique",
    "ufficio federale di statistica", "federal statistical office", "ofs", "bfs",
    "seco", "staatssekretariat für wirtschaft",
    "secrétariat d'état à l'économie", "secretariat d'etat a l'economie",
    "segretariato di stato dell'economia",
    "kanton zürich", "kanton zuerich", "canton de zurich", "cantone zurigo",
    "kanton luzern", "canton de lucerne", "cantone lucerna",
    "kanton", "canton", "cantone",
    "region", "regionen", "région", "regione",
    "bundesamt für", "bundesamt fuer",
    "bundesamt für bauten und logistik", "bundesamt fuer bauten und logistik", "bbl",
    "index", "verbraucherindex", "price index", "indice des prix", "indice dei prezzi",
    "wirtschaftsforschung", "economic research", "recherche économique",
    "recherche economique", "ricerca economica",
]

# ---------------- EXCLUSION KEYWORDS (DE/FR/IT/EN) ----------------
RAW_EXCLUDE = [
    "construction", "bau", "bâtiment", "batiment", "costruzioni",
    "health care", "gesundheit", "santé", "sante", "sanità", "sanita",
    "transport", "verkehr", "transports", "trasporti",
    "mobility", "mobilität", "mobilite", "mobilité", "mobilita",
    "sport",
    "culture", "kultur", "cultura",
    "street", "strasse", "straße", "rue", "strada",
    "infrastructure", "infrastruktur", "infrastruttura",
    "process", "prozess", "processus", "processo",
    "it", "informatik", "informatique", "informatica",
]

def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    return re.sub(r"\s+", " ", s.strip().lower())

INCLUDE = {_norm(k) for k in RAW_INCLUDE if k.strip()}
EXCLUDE = {_norm(k) for k in RAW_EXCLUDE if k.strip()}

# ---------------- Selenium setup ----------------
def make_driver(headless=True):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--no-sandbox"); opts.add_argument("--disable-gpu")
    opts.add_argument("--lang=de-DE")
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=opts)

def accept_cookies(driver):
    labels = ["Alle akzeptieren","Akzeptieren","Zustimmen","Einverstanden","OK","Okay","Ich stimme zu"]
    for label in labels:
        try:
            btn = WebDriverWait(driver, 4).until(
                EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{label}')]"))
            )
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)
            break
        except Exception:
            pass

def wait_for_table(driver, timeout=20):
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody tr"))
    )

def first_cell_token(driver):
    try:
        return driver.find_element(By.CSS_SELECTOR, "table tbody tr td").get_attribute("innerText").strip()
    except Exception:
        return None

def set_page_size_max(driver):
    tried = False
    for sel in ["select", "select.page-size", "select[name*='size' i]"]:
        try:
            el = driver.find_element(By.CSS_SELECTOR, sel)
            s = Select(el)
            labels = [o.text.strip() or o.get_attribute("value") for o in s.options]
            target = None
            for want in ("100","200","Alle","Alles"):
                for lbl in labels:
                    if want in lbl:
                        target = lbl; break
                if target: break
            if not target and labels: target = labels[-1]
            if target:
                old = first_cell_token(driver)
                s.select_by_visible_text(target)
                WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != old)
                tried = True
                break
        except Exception:
            pass
    if not tried:
        for want in ("100","200"):
            try:
                btn = driver.find_element(By.XPATH, f"//button[normalize-space()='{want}']")
                old = first_cell_token(driver)
                driver.execute_script("arguments[0].click();", btn)
                WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != old)
                break
            except Exception:
                pass

# ---------------- Full column extraction + filters ----------------
def unique_headers(headers):
    counts = defaultdict(int); out = []
    for h in headers:
        key = h or "col"; counts[key] += 1
        out.append(key if counts[key] == 1 else f"{key}_{counts[key]}")
    return out

def get_table_headers(driver):
    table = driver.find_element(By.CSS_SELECTOR, "table")
    ths = table.find_elements(By.CSS_SELECTOR, "thead th")
    headers = [th.get_attribute("innerText").strip() or th.get_attribute("title") or th.get_attribute("aria-label") or "" for th in ths]
    if not any(h for h in headers):
        try:
            first_row = table.find_element(By.CSS_SELECTOR, "tbody tr")
            tds = first_row.find_elements(By.CSS_SELECTOR, "td")
            tmp = []
            for td in tds:
                label = td.get_attribute("data-title") or td.get_attribute("aria-label") or ""
                tmp.append(label)
            headers = tmp
        except Exception:
            pass
    if not headers:
        first_row = table.find_element(By.CSS_SELECTOR, "tbody tr")
        n = len(first_row.find_elements(By.CSS_SELECTOR, "td"))
        headers = [f"col_{i+1}" for i in range(n)]
    return unique_headers([h.strip() for h in headers])

def row_dicts_all_columns(driver, headers):
    """
    Extract all visible columns and apply inclusion/exclusion filters.
    Adds <column>_link if a link exists in that cell.
    """
    table = driver.find_element(By.CSS_SELECTOR, "table")
    rows = []
    for tr in table.find_elements(By.CSS_SELECTOR, "tbody tr"):
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        if not tds: continue
        row = {}; concat = []
        for i, td in enumerate(tds):
            key = headers[i] if i < len(headers) else f"col_{i+1}"
            text = td.get_attribute("innerText").strip()
            row[key] = text
            concat.append(text)
            try:
                a = td.find_element(By.CSS_SELECTOR, "a[href]")
                row[f"{key}_link"] = a.get_attribute("href")
            except Exception:
                pass
        norm = _norm(" | ".join(concat))
        hits_incl = sorted({kw for kw in INCLUDE if kw in norm})
        hits_excl = sorted({kw for kw in EXCLUDE if kw in norm})
        if hits_incl and not hits_excl:
            row["matched_keywords"] = ", ".join(hits_incl)
            row["excluded_keywords_hit"] = ", ".join(hits_excl)  # should be empty on kept rows
            rows.append(row)
    return rows

def go_next(driver):
    try:
        nxt = driver.find_element(By.CSS_SELECTOR, "a[rel='next']")
        if (nxt.get_attribute("aria-disabled") or "").lower() == "true":
            return False
        token = first_cell_token(driver)
        driver.execute_script("arguments[0].click();", nxt)
        WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != token)
        return True
    except Exception:
        return False

# ---------------- Detail page scraping & PDF download ----------------
def selenium_cookies_to_requests(driver) -> requests.Session:
    s = requests.Session()
    for c in driver.get_cookies():
        s.cookies.set(c["name"], c["value"], domain=c.get("domain"))
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (compatible; EvergabeScraper/1.0)"
    })
    return s

def ensure_dir(path):
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)

def extract_detail_content_and_pdf(driver, url, req_session: requests.Session, timeout=25):
    """
    Open the detail URL with Selenium, scrape visible text content (main area),
    find the first PDF link, download it via requests (using Selenium cookies).
    Returns (detail_text, pdf_local_path or "")
    """
    try:
        driver.get(url)
        # wait for something meaningful to be present
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
        )
        # try to locate a main content area; fallback to all text
        selectors = [
            "main", "article", "#content", ".content", ".main", ".container",
            ".detail", ".panel-body"
        ]
        text = ""
        for sel in selectors:
            els = driver.find_elements(By.CSS_SELECTOR, sel)
            if els:
                text = els[0].get_attribute("innerText").strip()
                if len(text) > 100:  # looks like real content
                    break
        if not text:
            text = driver.find_element(By.TAG_NAME, "body").get_attribute("innerText").strip()
    except Exception:
        return "", ""

    # find first PDF link
    pdf_url = ""
    try:
        a_tags = driver.find_elements(By.CSS_SELECTOR, "a[href]")
        for a in a_tags:
            href = a.get_attribute("href") or ""
            if ".pdf" in href.lower():
                pdf_url = href
                break
    except Exception:
        pass

    pdf_path = ""
    if pdf_url:
        try:
            ensure_dir(DOWNLOAD_DIR)
            # make a deterministic filename from URL
            pdf_uid = uuid.uuid5(uuid.NAMESPACE_URL, pdf_url)
            dest = os.path.join(DOWNLOAD_DIR, f"{pdf_uid}.pdf")
            if not os.path.exists(dest):
                r = req_session.get(pdf_url, timeout=timeout)
                if r.ok and r.content[:4] in (b"%PDF",):  # naive PDF signature check
                    with open(dest, "wb") as f:
                        f.write(r.content)
                    pdf_path = dest
            else:
                pdf_path = dest
        except Exception:
            pdf_path = ""

    return text, pdf_path

# ---------------- Main crawl ----------------
def crawl_all_once(headless=True, max_pages=None, per_page_delay=0.15):
    driver = make_driver(headless=headless)
    driver.get(START_URL)
    accept_cookies(driver)
    wait_for_table(driver)
    set_page_size_max(driver)

    headers = get_table_headers(driver)
    results, page = [], 1
    while True:
        print(f"📄 Page {page} …", end="", flush=True)
        page_rows = row_dicts_all_columns(driver, headers)
        results.extend(page_rows)
        print(f" kept {len(page_rows)}")
        if max_pages and page >= max_pages:
            break
        time.sleep(per_page_delay)
        if not go_next(driver):
            break
        page += 1

    # Build unique ids now (stable by URL+Bezeichnung fallback)
    for idx, r in enumerate(results, 1):
        base = r.get("detail_url") or r.get("Bezeichnung") or str(idx)
        # stable UUID v5 (same input -> same uid)
        r["uid"] = "EVG-" + uuid.uuid5(uuid.NAMESPACE_URL, base).hex[:12].upper()

    # Use requests with Selenium cookies for downloads
    req_sess = selenium_cookies_to_requests(driver)

    # Visit each detail_url, collect content + pdf (first found)
    for i, r in enumerate(results, 1):
        url = r.get("detail_url", "")
        if not url:
            r["detail_content"] = ""
            r["pdf_path"] = ""
            continue
        print(f"   ↪ details [{i}/{len(results)}]: {url}")
        text, pdf_local = extract_detail_content_and_pdf(driver, url, req_sess)
        r["detail_content"] = text
        r["pdf_path"] = pdf_local

    driver.quit()

    # Save CSV
    all_keys = set(headers + [f"{h}_link" for h in headers] + ["matched_keywords","excluded_keywords_hit","uid","detail_content","pdf_path"])
    for r in results: all_keys.update(r.keys())
    preferred = ["uid","Bezeichnung","Geschäftszeichen","Vergabestelle","Ort der Leistung","matched_keywords","detail_url","pdf_path","detail_content"]
    header = [k for k in preferred if k in all_keys] + [k for k in sorted(all_keys) if k not in preferred]

    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    out = f"evergabe_filtered_with_details_{ts}.csv"
    with open(out, "w", newline="", encoding="utf-8-sig") as f:
        w = csv.DictWriter(f, fieldnames=header)
        w.writeheader(); w.writerows(results)

    print(f"✅ Saved {len(results)} rows to {out}")
    print(f"📂 PDFs (if found) are in: {os.path.abspath(DOWNLOAD_DIR)}")

if __name__ == "__main__":
    # headless=False to watch; set max_pages for a quick test
    crawl_all_once(headless=True, max_pages=None, per_page_delay=0.15)


📄 Page 1 … kept 2
📄 Page 2 … kept 5
📄 Page 3 … kept 2
📄 Page 4 … kept 2
📄 Page 5 … kept 2
📄 Page 6 … kept 1
📄 Page 7 … kept 1
📄 Page 8 … kept 1
📄 Page 9 … kept 3
📄 Page 10 … kept 2
📄 Page 11 … kept 1
📄 Page 12 … kept 2
📄 Page 13 … kept 0
📄 Page 14 … kept 2
📄 Page 15 … kept 2
📄 Page 16 … kept 2
📄 Page 17 … kept 4
📄 Page 18 … kept 4
📄 Page 19 … kept 5
📄 Page 20 … kept 2
📄 Page 21 … kept 2
📄 Page 22 … kept 4
📄 Page 23 … kept 1
📄 Page 24 … kept 3
📄 Page 25 … kept 2
📄 Page 26 … kept 3
📄 Page 27 … kept 1
📄 Page 28 … kept 1
📄 Page 29 … kept 3
📄 Page 30 … kept 1
📄 Page 31 … kept 1
📄 Page 32 … kept 0
📄 Page 33 … kept 3
📄 Page 34 … kept 4
📄 Page 35 … kept 1
📄 Page 36 … kept 2
📄 Page 37 … kept 1
📄 Page 38 … kept 1
📄 Page 39 … kept 2
📄 Page 40 … kept 2
📄 Page 41 … kept 3
📄 Page 42 … kept 3
📄 Page 43 … kept 1
📄 Page 44 … kept 2
📄 Page 45 … kept 1
📄 Page 46 … kept 3
📄 Page 47 … kept 4
📄 Page 48 … kept 0
📄 Page 49 … kept 2
📄 Page 50 … kept 0
📄 Page 51 … kept 2
📄 Page 52 … kept 0
📄 Page 53 … kept 4
📄 

📄 Page 1 … kept 3
📄 Page 2 … kept 4
📄 Page 3 … kept 2
📄 Page 4 … kept 3
📄 Page 5 … kept 1
📄 Page 6 … kept 1
📄 Page 7 … kept 2
📄 Page 8 … kept 1
📄 Page 9 … kept 3
📄 Page 10 … kept 1
📄 Page 11 … kept 1
📄 Page 12 … kept 2
📄 Page 13 … kept 0
📄 Page 14 … kept 3
📄 Page 15 … kept 1
📄 Page 16 … kept 2
📄 Page 17 … kept 5
📄 Page 18 … kept 5
📄 Page 19 … kept 4
📄 Page 20 … kept 1
📄 Page 21 … kept 2
📄 Page 22 … kept 4
📄 Page 23 … kept 1
📄 Page 24 … kept 3
📄 Page 25 … kept 2
📄 Page 26 … kept 3
📄 Page 27 … kept 2
📄 Page 28 … kept 1
📄 Page 29 … kept 2
📄 Page 30 … kept 2
📄 Page 31 … kept 0
📄 Page 32 … kept 1
📄 Page 33 … kept 3
📄 Page 34 … kept 4
📄 Page 35 … kept 0
📄 Page 36 … kept 2
📄 Page 37 … kept 1
📄 Page 38 … kept 2
📄 Page 39 … kept 2
📄 Page 40 … kept 1
📄 Page 41 … kept 4
📄 Page 42 … kept 2
📄 Page 43 … kept 1
📄 Page 44 … kept 2
📄 Page 45 … kept 2
📄 Page 46 … kept 3
📄 Page 47 … kept 3
📄 Page 48 … kept 0
📄 Page 49 … kept 2
📄 Page 50 … kept 0
📄 Page 51 … kept 2
📄 Page 52 … kept 1
📄 Page 53 … kept 4
📄 

In [ ]:
# evergabe_singlepass_with_details_and_pdfs.py
# FAST single-pass crawler with:
# - multilingual inclusion/exclusion filters
# - full column extraction (+ per-cell links)
# - per-row UID
# - detail page scraping (detail_content)
# - download ALL PDFs per row -> downloads/<uid>_N.pdf

import csv, time, re, math, datetime, unicodedata, os, uuid, pathlib
from collections import defaultdict
from urllib.parse import urljoin

import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

START_URL = "https://www.evergabe-online.de/search.html?4"
DOWNLOAD_DIR = "downloads"  # PDFs saved here

# ---------------- Inclusion / Exclusion keywords (DE/FR/IT/EN) ----------------
RAW_INCLUDE = [
    "study","studie","étude","etude","studio",
    "analysis","analyse","analisi",
    "economy","economics","wirtschaft","ökonomie","economie","economia",
    "benchmark","benchmarking",
    "wirtschaftsberatung","consulting","conseil","consulenza",
    "72000000","79300000",
    "it-dienste",
    "beratung, software-entwicklung, internet und hilfestellung",
    "markt- und wirtschaftsforschung","umfragen und statistiken",
    "dienstleistung","service","prestation","prestazione",
    "ausschreibung","appel d'offres","appels d'offres","gara","procurement",
    "offenes verfahren","procédure ouverte","procedure ouverte","procedura aperta",
    "bks","bundesamt für statistik","office fédéral de la statistique","office federal de la statistique",
    "ufficio federale di statistica","federal statistical office","ofs","bfs",
    "seco","staatssekretariat für wirtschaft",
    "secrétariat d'état à l'économie","secretariat d'etat a l'economie",
    "segretariato di stato dell'economia",
    "kanton zürich","kanton zuerich","canton de zurich","cantone zurigo",
    "kanton luzern","canton de lucerne","cantone lucerna",
    "kanton","canton","cantone",
    "region","regionen","région","regione",
    "bundesamt für","bundesamt fuer",
    "bundesamt für bauten und logistik","bundesamt fuer bauten und logistik","bbl",
    "index","verbraucherindex","price index","indice des prix","indice dei prezzi",
    "wirtschaftsforschung","economic research","recherche économique","recherche economique","ricerca economica",
]
RAW_EXCLUDE = [
    "construction","bau","bâtiment","batiment","costruzioni",
    "health care","gesundheit","santé","sante","sanità","sanita",
    "transport","verkehr","transports","trasporti",
    "mobility","mobilität","mobilite","mobilité","mobilita",
    "sport",
    "culture","kultur","cultura",
    "street","strasse","straße","rue","strada",
    "infrastructure","infrastruktur","infrastruttura",
    "process","prozess","processus","processo",
    "it","informatik","informatique","informatica",
]

def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    return re.sub(r"\s+", " ", s.strip().lower())

INCLUDE = {_norm(k) for k in RAW_INCLUDE if k.strip()}
EXCLUDE = {_norm(k) for k in RAW_EXCLUDE if k.strip()}

# ---------------- Selenium setup ----------------
def make_driver(headless=True):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--no-sandbox"); opts.add_argument("--disable-gpu")
    opts.add_argument("--lang=de-DE")
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=opts)

def accept_cookies(driver):
    labels = ["Alle akzeptieren","Akzeptieren","Zustimmen","Einverstanden","OK","Okay","Ich stimme zu"]
    for label in labels:
        try:
            btn = WebDriverWait(driver, 4).until(
                EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{label}')]"))
            )
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)
            break
        except Exception:
            pass

def wait_for_table(driver, timeout=20):
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody tr"))
    )

def first_cell_token(driver):
    try:
        return driver.find_element(By.CSS_SELECTOR, "table tbody tr td").get_attribute("innerText").strip()
    except Exception:
        return None

def set_page_size_max(driver):
    tried = False
    # Try <select>
    for sel in ["select", "select.page-size", "select[name*='size' i]"]:
        try:
            el = driver.find_element(By.CSS_SELECTOR, sel)
            s = Select(el)
            labels = [o.text.strip() or o.get_attribute("value") for o in s.options]
            target = None
            for want in ("100","200","Alle","Alles"):
                for lbl in labels:
                    if want in lbl:
                        target = lbl; break
                if target: break
            if not target and labels: target = labels[-1]
            if target:
                old = first_cell_token(driver)
                s.select_by_visible_text(target)
                WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != old)
                tried = True
                break
        except Exception:
            pass
    # Try buttons
    if not tried:
        for want in ("100","200"):
            try:
                btn = driver.find_element(By.XPATH, f"//button[normalize-space()='{want}']")
                old = first_cell_token(driver)
                driver.execute_script("arguments[0].click();", btn)
                WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != old)
                break
            except Exception:
                pass

# ---------------- Full column extraction + filters ----------------
def unique_headers(headers):
    counts = defaultdict(int); out = []
    for h in headers:
        key = h or "col"; counts[key] += 1
        out.append(key if counts[key] == 1 else f"{key}_{counts[key]}")
    return out

def get_table_headers(driver):
    table = driver.find_element(By.CSS_SELECTOR, "table")
    ths = table.find_elements(By.CSS_SELECTOR, "thead th")
    headers = [th.get_attribute("innerText").strip() or th.get_attribute("title") or th.get_attribute("aria-label") or "" for th in ths]
    if not any(h for h in headers):
        try:
            first_row = table.find_element(By.CSS_SELECTOR, "tbody tr")
            tds = first_row.find_elements(By.CSS_SELECTOR, "td")
            tmp = []
            for td in tds:
                label = td.get_attribute("data-title") or td.get_attribute("aria-label") or ""
                tmp.append(label)
            headers = tmp
        except Exception:
            pass
    if not headers:
        first_row = table.find_element(By.CSS_SELECTOR, "tbody tr")
        n = len(first_row.find_elements(By.CSS_SELECTOR, "td"))
        headers = [f"col_{i+1}" for i in range(n)]
    return unique_headers([h.strip() for h in headers])

def row_dicts_all_columns(driver, headers):
    table = driver.find_element(By.CSS_SELECTOR, "table")
    rows = []
    for tr in table.find_elements(By.CSS_SELECTOR, "tbody tr"):
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        if not tds: continue
        row = {}; concat = []
        for i, td in enumerate(tds):
            key = headers[i] if i < len(headers) else f"col_{i+1}"
            text = td.get_attribute("innerText").strip()
            row[key] = text
            concat.append(text)
            try:
                a = td.find_element(By.CSS_SELECTOR, "a[href]")
                row[f"{key}_link"] = a.get_attribute("href")
            except Exception:
                pass
        norm = _norm(" | ".join(concat))
        hits_incl = sorted({kw for kw in INCLUDE if kw in norm})
        hits_excl = sorted({kw for kw in EXCLUDE if kw in norm})
        if hits_incl and not hits_excl:
            row["matched_keywords"] = ", ".join(hits_incl)
            rows.append(row)
    return rows

def go_next(driver):
    try:
        nxt = driver.find_element(By.CSS_SELECTOR, "a[rel='next']")
        if (nxt.get_attribute("aria-disabled") or "").lower() == "true":
            return False
        token = first_cell_token(driver)
        driver.execute_script("arguments[0].click();", nxt)
        WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != token)
        return True
    except Exception:
        return False

# ---------------- Detail page scraping & PDFs ----------------
def ensure_dir(path):
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)

def selenium_cookies_to_requests(driver) -> requests.Session:
    s = requests.Session()
    for c in driver.get_cookies():
        s.cookies.set(c["name"], c["value"], domain=c.get("domain"))
    s.headers.update({"User-Agent": "Mozilla/5.0 (compatible; EvergabeScraper/1.0)"})
    return s

def collect_pdf_links(driver, base_url):
    """Find ALL candidate PDF links on the current page."""
    found = set()
    # direct anchors with .pdf
    for a in driver.find_elements(By.CSS_SELECTOR, "a[href]"):
        href = a.get_attribute("href") or ""
        text = (a.text or "").lower()
        typ = (a.get_attribute("type") or "").lower()
        if ".pdf" in href.lower() or "pdf" in text or "application/pdf" in typ:
            found.add(urljoin(base_url, href))
    # also look in <embed> or <object>
    for e in driver.find_elements(By.CSS_SELECTOR, "embed[src], object[data]"):
        src = e.get_attribute("src") or e.get_attribute("data") or ""
        if ".pdf" in src.lower():
            found.add(urljoin(base_url, src))
    return sorted(found)

def click_possible_docs_tabs(driver):
    """Try to reveal documents area (common tab labels)."""
    labels = ["Dokumente","Vergabeunterlagen","Unterlagen","Documents","Documenti","Documents"]
    for lab in labels:
        try:
            el = driver.find_element(By.XPATH, f"//a[contains(., '{lab}') or //button[contains(., '{lab}')]]")
            driver.execute_script("arguments[0].click();", el)
            time.sleep(0.5)
        except Exception:
            pass

def fetch_detail_and_pdfs(driver, detail_url, uid, req_sess, max_pdfs=10, timeout=30):
    """Open detail page, extract text and download ALL PDFs. Returns (text, [paths])."""
    driver.get(detail_url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))
    # try to open doc tabs if any
    click_possible_docs_tabs(driver)
    time.sleep(0.5)

    # collect page text
    text = ""
    for sel in ["main", "article", "#content", ".content", ".main", ".container", ".detail", ".panel-body"]:
        els = driver.find_elements(By.CSS_SELECTOR, sel)
        if els:
            t = els[0].get_attribute("innerText").strip()
            if len(t) > 80:
                text = t; break
    if not text:
        text = driver.find_element(By.TAG_NAME, "body").get_attribute("innerText").strip()

    # collect & download PDFs
    pdf_urls = collect_pdf_links(driver, driver.current_url)
    ensure_dir(DOWNLOAD_DIR)
    paths = []
    for i, purl in enumerate(pdf_urls[:max_pdfs], 1):
        try:
            # request with Referer
            r = req_sess.get(purl, headers={"Referer": detail_url}, timeout=timeout, allow_redirects=True)
            ctype = (r.headers.get("Content-Type") or "").lower()
            is_pdf = r.ok and (b"%PDF" in r.content[:1024] or "application/pdf" in ctype)
            if is_pdf:
                dest = os.path.join(DOWNLOAD_DIR, f"{uid}_{i}.pdf")
                with open(dest, "wb") as f:
                    f.write(r.content)
                paths.append(dest)
        except Exception:
            continue

    return text, paths

# ---------------- Master crawl ----------------
def crawl_all_once(headless=True, max_pages=None, per_page_delay=0.15):
    driver = make_driver(headless=headless)
    driver.get(START_URL)
    accept_cookies(driver)
    wait_for_table(driver)
    set_page_size_max(driver)

    headers = get_table_headers(driver)
    rows, page = [], 1
    while True:
        print(f"📄 Page {page} …", end="", flush=True)
        page_rows = row_dicts_all_columns(driver, headers)
        rows.extend(page_rows)
        print(f" kept {len(page_rows)}")
        if max_pages and page >= max_pages:
            break
        time.sleep(per_page_delay)
        if not go_next(driver):
            break
        page += 1

    # Assign stable UID per row (based on detail_url fallback to Bezeichnung)
    for idx, r in enumerate(rows, 1):
        base = r.get("detail_url") or r.get("Bezeichnung") or str(idx)
        r["uid"] = "EVG-" + uuid.uuid5(uuid.NAMESPACE_URL, base).hex[:12].upper()

    # Prepare requests session with Selenium cookies
    req_sess = selenium_cookies_to_requests(driver)

    # Visit each detail, extract content, download ALL PDFs
    for i, r in enumerate(rows, 1):
        durl = r.get("detail_url", "")
        if not durl:
            r["detail_content"] = ""
            r["pdf_paths"] = ""
            continue
        print(f"   ↪ details [{i}/{len(rows)}]: {durl}")
        text, pdfs = fetch_detail_and_pdfs(driver, durl, r["uid"], req_sess)
        r["detail_content"] = text
        r["pdf_paths"] = " | ".join(pdfs) if pdfs else ""

    driver.quit()

    # Save CSV
    all_keys = set(headers + [f"{h}_link" for h in headers] + ["uid","matched_keywords","detail_url","detail_content","pdf_paths"])
    for r in rows: all_keys.update(r.keys())
    preferred = ["uid","Bezeichnung","Geschäftszeichen","Vergabestelle","Ort der Leistung",
                 "matched_keywords","detail_url","pdf_paths","detail_content"]
    header = [k for k in preferred if k in all_keys] + [k for k in sorted(all_keys) if k not in preferred]

    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    out = f"evergabe_filtered_with_pdfs_{ts}.csv"
    with open(out, "w", newline="", encoding="utf-8-sig") as f:
        w = csv.DictWriter(f, fieldnames=header)
        w.writeheader(); w.writerows(rows)

    print(f"✅ Saved {len(rows)} rows to {out}")
    print(f"📂 PDFs saved under: {os.path.abspath(DOWNLOAD_DIR)}")

if __name__ == "__main__":
    # headless=False to watch; set max_pages for quick test
    crawl_all_once(headless=True, max_pages=None, per_page_delay=0.15)


In [14]:
pip install selenium webdriver-manager requests

Note: you may need to restart the kernel to use updated packages.


PDV+CSV

In [17]:
# evergabe_singlepass_full.py
# Single-pass, fast scraper for evergabe-online.de:
# - Multilingual inclusion & exclusion keyword filters
# - Full column extraction (+ per-cell link URLs)
# - Stable UID per row
# - Visits each detail_url, scrapes page content, downloads ALL PDFs
# - Saves CSV + PDFs under ./downloads

import csv
import time
import re
import math
import datetime
import unicodedata
import os
import uuid
import pathlib
from collections import defaultdict
from urllib.parse import urljoin

import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# -------- Settings --------
START_URL = "https://www.evergabe-online.de/search.html?4"
DOWNLOAD_DIR = "downloads"     # PDFs will be saved here
HEADLESS = True                # set to False to watch the browser
MAX_PAGES = None               # set an int (e.g., 5) for quick testing
PER_PAGE_DELAY = 0.15          # seconds; be polite but fast

# -------- Keywords (DE/FR/IT/EN) --------
RAW_INCLUDE = [
    # Study / Studie
    "study","studie","étude","etude","studio",
    # Analysis / Analyse
    "analysis","analyse","analisi",
    # Economy / Wirtschaft / Ökonomie
    "economy","economics","wirtschaft","ökonomie","economie","economia",
    # Benchmarking
    "benchmark","benchmarking",
    # Wirtschaftsberatung / consulting
    "wirtschaftsberatung","consulting","conseil","consulenza",
    # CPV & categories
    "72000000","79300000",
    "it-dienste",
    "beratung, software-entwicklung, internet und hilfestellung",
    "markt- und wirtschaftsforschung","umfragen und statistiken",
    # Dienstleistung / services
    "dienstleistung","service","prestation","prestazione",
    # Ausschreibung / tender
    "ausschreibung","appel d'offres","appels d'offres","gara","procurement",
    # Offenes Verfahren / open procedure
    "offenes verfahren","procédure ouverte","procedure ouverte","procedura aperta",
    # BKS / Bundesamt für Statistik / OFS
    "bks","bundesamt für statistik","office fédéral de la statistique",
    "office federal de la statistique","ufficio federale di statistica",
    "federal statistical office","ofs","bfs",
    # SECO
    "seco","staatssekretariat für wirtschaft",
    "secrétariat d'état à l'économie","secretariat d'etat a l'economie",
    "segretariato di stato dell'economia",
    # Cantons / regions
    "kanton zürich","kanton zuerich","canton de zurich","cantone zurigo",
    "kanton luzern","canton de lucerne","cantone lucerna",
    "kanton","canton","cantone",
    "region","regionen","région","regione",
    # Bundesamt für.. (generic)
    "bundesamt für","bundesamt fuer",
    # BBL
    "bundesamt für bauten und logistik","bundesamt fuer bauten und logistik","bbl",
    # Index
    "index","verbraucherindex","price index","indice des prix","indice dei prezzi",
    # Wirtschaftsforschung
    "wirtschaftsforschung","economic research","recherche économique",
    "recherche economique","ricerca economica",
]

RAW_EXCLUDE = [
    "construction","bau","bâtiment","batiment","costruzioni",
    "health care","gesundheit","santé","sante","sanità","sanita",
    "transport","verkehr","transports","trasporti",
    "mobility","mobilität","mobilite","mobilité","mobilita",
    "sport",
    "culture","kultur","cultura",
    "street","strasse","straße","rue","strada",
    "infrastructure","infrastruktur","infrastruttura",
    "process","prozess","processus","processo",
    "it","informatik","informatique","informatica",
]

def _norm(s: str) -> str:
    """Lowercase, strip accents, collapse whitespace."""
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    return re.sub(r"\s+", " ", s.strip().lower())

INCLUDE = {_norm(k) for k in RAW_INCLUDE if k.strip()}
EXCLUDE = {_norm(k) for k in RAW_EXCLUDE if k.strip()}

# -------- Selenium setup --------
def make_driver(headless=True):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--lang=de-DE")
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=opts)

def accept_cookies(driver):
    labels = ["Alle akzeptieren","Akzeptieren","Zustimmen","Einverstanden","OK","Okay","Ich stimme zu"]
    for label in labels:
        try:
            btn = WebDriverWait(driver, 4).until(
                EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{label}')]"))
            )
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)
            break
        except Exception:
            pass

def wait_for_table(driver, timeout=25):
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody tr"))
    )

def first_cell_token(driver):
    try:
        return driver.find_element(By.CSS_SELECTOR, "table tbody tr td").get_attribute("innerText").strip()
    except Exception:
        return None

def set_page_size_max(driver):
    """Try to switch page size to the largest (100/200/Alle)."""
    tried = False
    for sel in ["select", "select.page-size", "select[name*='size' i]"]:
        try:
            el = driver.find_element(By.CSS_SELECTOR, sel)
            s = Select(el)
            labels = [o.text.strip() or o.get_attribute("value") for o in s.options]
            target = None
            for want in ("200","100","Alle","Alles"):
                for lbl in labels:
                    if want in (lbl or ""):
                        target = lbl; break
                if target: break
            if not target and labels:
                target = labels[-1]
            if target:
                old = first_cell_token(driver)
                s.select_by_visible_text(target)
                WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != old)
                tried = True
                break
        except Exception:
            pass
    if not tried:
        for want in ("200","100"):
            try:
                btn = driver.find_element(By.XPATH, f"//button[normalize-space()='{want}']")
                old = first_cell_token(driver)
                driver.execute_script("arguments[0].click();", btn)
                WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != old)
                break
            except Exception:
                pass

# -------- Table header & row extraction --------
def unique_headers(headers):
    counts = defaultdict(int)
    out = []
    for h in headers:
        key = h or "col"
        counts[key] += 1
        out.append(key if counts[key] == 1 else f"{key}_{counts[key]}")
    return out

def get_table_headers(driver):
    table = driver.find_element(By.CSS_SELECTOR, "table")
    ths = table.find_elements(By.CSS_SELECTOR, "thead th")
    headers = [th.get_attribute("innerText").strip() or th.get_attribute("title") or th.get_attribute("aria-label") or "" for th in ths]
    if not any(h for h in headers):
        # fall back to data-title/aria-label on tds
        try:
            first_row = table.find_element(By.CSS_SELECTOR, "tbody tr")
            tds = first_row.find_elements(By.CSS_SELECTOR, "td")
            tmp = []
            for td in tds:
                label = td.get_attribute("data-title") or td.get_attribute("aria-label") or ""
                tmp.append(label)
            headers = tmp
        except Exception:
            pass
    if not headers:
        first_row = table.find_element(By.CSS_SELECTOR, "tbody tr")
        n = len(first_row.find_elements(By.CSS_SELECTOR, "td"))
        headers = [f"col_{i+1}" for i in range(n)]
    return unique_headers([h.strip() for h in headers])

def row_dicts_all_columns(driver, headers):
    """
    Extract all visible columns and apply inclusion/exclusion filters.
    Adds <column>_link if a link exists in that cell.
    """
    table = driver.find_element(By.CSS_SELECTOR, "table")
    rows = []
    for tr in table.find_elements(By.CSS_SELECTOR, "tbody tr"):
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        if not tds:
            continue
        row = {}
        concat = []
        for i, td in enumerate(tds):
            key = headers[i] if i < len(headers) else f"col_{i+1}"
            text = td.get_attribute("innerText").strip()
            row[key] = text
            concat.append(text)
            # capture first link in cell
            try:
                a = td.find_element(By.CSS_SELECTOR, "a[href]")
                row[f"{key}_link"] = a.get_attribute("href")
                if i == 0 and "detail_url" not in row:
                    row["detail_url"] = a.get_attribute("href")  # likely the main detail link
            except Exception:
                pass
        norm = _norm(" | ".join(concat))
        hits_incl = sorted({kw for kw in INCLUDE if kw in norm})
        hits_excl = sorted({kw for kw in EXCLUDE if kw in norm})
        if hits_incl and not hits_excl:
            row["matched_keywords"] = ", ".join(hits_incl)
            rows.append(row)
    return rows

def go_next(driver):
    try:
        nxt = driver.find_element(By.CSS_SELECTOR, "a[rel='next']")
        if (nxt.get_attribute("aria-disabled") or "").lower() == "true":
            return False
        token = first_cell_token(driver)
        driver.execute_script("arguments[0].click();", nxt)
        WebDriverWait(driver, 15).until(lambda d: first_cell_token(d) != token)
        return True
    except Exception:
        return False

# -------- Detail page scraping & PDFs --------
def ensure_dir(path):
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)

def selenium_cookies_to_requests(driver) -> requests.Session:
    s = requests.Session()
    for c in driver.get_cookies():
        s.cookies.set(c["name"], c["value"], domain=c.get("domain"))
    s.headers.update({"User-Agent": "Mozilla/5.0 (compatible; EvergabeScraper/1.0)"})
    return s

def collect_pdf_links(driver, base_url):
    """Find ALL candidate PDF links on the current page."""
    found = set()
    # direct anchors with .pdf or type
    for a in driver.find_elements(By.CSS_SELECTOR, "a[href]"):
        href = a.get_attribute("href") or ""
        text = (a.text or "").lower()
        typ = (a.get_attribute("type") or "").lower()
        if ".pdf" in href.lower() or "pdf" in text or "application/pdf" in typ:
            found.add(urljoin(base_url, href))
    # <embed> or <object>
    for e in driver.find_elements(By.CSS_SELECTOR, "embed[src], object[data]"):
        src = e.get_attribute("src") or e.get_attribute("data") or ""
        if ".pdf" in (src or "").lower():
            found.add(urljoin(base_url, src))
    return sorted(found)

def click_possible_docs_tabs(driver):
    """Try to reveal document tabs/sections."""
    labels = ["Dokumente","Vergabeunterlagen","Unterlagen","Documents","Documenti","Documents","Downloads"]
    for lab in labels:
        try:
            el = driver.find_element(By.XPATH, f"//a[contains(., '{lab}')]|//button[contains(., '{lab}')]")
            driver.execute_script("arguments[0].click();", el)
            time.sleep(0.5)
        except Exception:
            pass

def fetch_detail_and_pdfs(driver, detail_url, uid, req_sess, max_pdfs=10, timeout=30):
    """Open detail page, extract visible text, download ALL PDFs. Returns (text, [paths])."""
    try:
        driver.get(detail_url)
        WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))
        time.sleep(0.8)  # let JS render if needed
    except Exception:
        return "", []

    # Open document tabs if any
    click_possible_docs_tabs(driver)
    time.sleep(0.3)

    # Extract main text
    text = ""
    for sel in ["main", "article", "#content", ".content", ".main", ".container", ".detail", ".panel-body", ".modul-detailansicht"]:
        els = driver.find_elements(By.CSS_SELECTOR, sel)
        if els:
            t = els[0].get_attribute("innerText").strip()
            if len(t) > 50:
                text = t
                break
    if not text:
        try:
            text = driver.find_element(By.TAG_NAME, "body").get_attribute("innerText").strip()
        except Exception:
            text = ""

    # Collect & download PDFs
    pdf_urls = collect_pdf_links(driver, driver.current_url)
    ensure_dir(DOWNLOAD_DIR)
    paths = []
    for i, purl in enumerate(pdf_urls[:max_pdfs], 1):
        try:
            r = req_sess.get(purl, headers={"Referer": detail_url}, timeout=timeout, allow_redirects=True)
            ctype = (r.headers.get("Content-Type") or "").lower()
            is_pdf = r.ok and (b"%PDF" in r.content[:1024] or "application/pdf" in ctype)
            if is_pdf:
                dest = os.path.join(DOWNLOAD_DIR, f"{uid}_{i}.pdf")
                with open(dest, "wb") as f:
                    f.write(r.content)
                paths.append(dest)
        except Exception:
            continue

    return text, paths

# -------- Master crawl --------
def crawl_all_once(headless=HEADLESS, max_pages=MAX_PAGES, per_page_delay=PER_PAGE_DELAY):
    driver = make_driver(headless=headless)
    driver.get(START_URL)
    accept_cookies(driver)
    wait_for_table(driver)
    set_page_size_max(driver)

    # Build headers once (fastest for consistent table)
    headers = get_table_headers(driver)

    rows = []
    page = 1
    while True:
        print(f"📄 Page {page} …", end="", flush=True)
        page_rows = row_dicts_all_columns(driver, headers)
        rows.extend(page_rows)
        print(f" kept {len(page_rows)}")
        if max_pages and page >= max_pages:
            break
        time.sleep(per_page_delay)
        if not go_next(driver):
            break
        page += 1

    # Assign stable UID per row (based on detail_url, fallback to Bezeichnung)
    for idx, r in enumerate(rows, 1):
        base = r.get("detail_url") or r.get("Bezeichnung") or str(idx)
        r["uid"] = "EVG-" + uuid.uuid5(uuid.NAMESPACE_URL, base).hex[:12].upper()

    # Prepare requests session with Selenium cookies (for downloads)
    req_sess = selenium_cookies_to_requests(driver)

    # Visit each detail_url, extract content, download PDFs
    total = len(rows)
    for i, r in enumerate(rows, 1):
        durl = r.get("detail_url", "")
        if not durl:
            r["detail_content"] = ""
            r["pdf_paths"] = ""
            print(f"   ↪ [{i}/{total}] no detail_url")
            continue
        print(f"   ↪ [{i}/{total}] details: {durl}")
        text, pdfs = fetch_detail_and_pdfs(driver, durl, r["uid"], req_sess)
        r["detail_content"] = text
        r["pdf_paths"] = " | ".join(pdfs) if pdfs else ""

    driver.quit()

    # Save CSV
    all_keys = set(headers + [f"{h}_link" for h in headers] + ["uid","matched_keywords","detail_url","detail_content","pdf_paths"])
    for r in rows:
        all_keys.update(r.keys())

    preferred = [
        "uid","Bezeichnung","Geschäftszeichen","Vergabestelle","Ort der Leistung",
        "matched_keywords","detail_url","pdf_paths","detail_content"
    ]
    header = [k for k in preferred if k in all_keys] + [k for k in sorted(all_keys) if k not in preferred]

    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    out = f"evergabe_filtered_with_details_{ts}.csv"
    with open(out, "w", newline="", encoding="utf-8-sig") as f:
        w = csv.DictWriter(f, fieldnames=header)
        w.writeheader()
        w.writerows(rows)

    print(f"\n✅ Saved {len(rows)} rows to {out}")
    print(f"📂 PDFs saved to: {os.path.abspath(DOWNLOAD_DIR)}")

if __name__ == "__main__":
    crawl_all_once()


📄 Page 1 … kept 2
📄 Page 2 … kept 5
📄 Page 3 … kept 2
📄 Page 4 … kept 2
📄 Page 5 … kept 2
📄 Page 6 … kept 1
📄 Page 7 … kept 1
📄 Page 8 … kept 1
📄 Page 9 … kept 3
📄 Page 10 … kept 2
📄 Page 11 … kept 1
📄 Page 12 … kept 2
📄 Page 13 … kept 0
📄 Page 14 … kept 2
📄 Page 15 … kept 2
📄 Page 16 … kept 2
📄 Page 17 … kept 4
📄 Page 18 … kept 4
📄 Page 19 … kept 5
📄 Page 20 … kept 2
📄 Page 21 … kept 2
📄 Page 22 … kept 4
📄 Page 23 … kept 1
📄 Page 24 … kept 3
📄 Page 25 … kept 2
📄 Page 26 … kept 3
📄 Page 27 … kept 1
📄 Page 28 … kept 1
📄 Page 29 … kept 3
📄 Page 30 … kept 1
📄 Page 31 … kept 1
📄 Page 32 … kept 0
📄 Page 33 … kept 3
📄 Page 34 … kept 4
📄 Page 35 … kept 1
📄 Page 36 … kept 2
📄 Page 37 … kept 1
📄 Page 38 … kept 1
📄 Page 39 … kept 2
📄 Page 40 … kept 2
📄 Page 41 … kept 3
📄 Page 42 … kept 3
📄 Page 43 … kept 1
📄 Page 44 … kept 2
📄 Page 45 … kept 1
📄 Page 46 … kept 3
📄 Page 47 … kept 4
📄 Page 48 … kept 0
📄 Page 49 … kept 2
📄 Page 50 … kept 0
📄 Page 51 … kept 2
📄 Page 52 … kept 0
📄 Page 53 … kept 4
📄 